In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('data_small.csv')
print(df.shape)

(289551, 17)


In [5]:
df.head()

,id,campaignName,description,categories,duration,monetaryGoal,country,city,state,date_created,deadline_date,launched_date,state_changed_at,backers_count,usd_pledged,url,target
0,1123889576,mars-on-earth-an-art-residency,Help a fine art photographer continue her proj...,Space Exploration,27,1000,US,Boston,MA,2015-06-24,2015-10-23,2015-09-26,2015-10-23,53,1884,https://www.kickstarter.com/projects/cassandra...,successful
1,1724173143,vulcan-i-rocket-powered-by-3d-printed-engine,Team of undergraduates racing to be the first ...,Space Exploration,30,15000,US,San Diego,CA,2014-05-06,2015-05-21,2015-04-21,2015-05-21,465,21882,https://www.kickstarter.com/projects/105499101...,successful
2,707260502,starscraper-the-next-generation-of-suborbital-...,What if we built a rocket that is better than ...,Space Exploration,31,10000,US,Boston,MA,2014-11-29,2015-01-09,2014-12-09,2015-01-09,294,17176,https://www.kickstarter.com/projects/burpg/sta...,successful
3,497637964,students-building-a-near-space-balloon-with-li...,A group of high school students are building a...,Space Exploration,30,150,US,Mountain View,CA,2014-11-19,2015-11-26,2015-10-27,2015-11-26,45,970,https://www.kickstarter.com/projects/136362214...,successful
4,1546008758,earth-360,Re-inventing the way we look at our planet by ...,Space Exploration,30,7500,US,Fairfield,CT,2012-04-11,2012-09-21,2012-08-22,2012-09-21,28,7576,https://www.kickstarter.com/projects/211370922...,successful


## Drop Duplicates

In [ ]:
df2 = df.drop_duplicates()
df2.shape

In [ ]:
# write to csv
# df.to_csv('clean_data_no_dup.csv', index=False)

In [82]:
# see how many total campaigns there are

df4 = df3.drop_duplicates(subset=['name', 'blurb'])

In [83]:
df4.shape[0], df3.shape[0]

(288267, 649853)

## EDA

In [87]:
unique_campaigns = df4.shape[0]
total_rows = df3.shape[0]
print(f'There are {total_rows} in the dataset')
print(f'There are {unique_campaigns} in the dataset')

There are 649853 in the dataset
There are 288267 in the dataset


## Encode Strings

In [6]:
def create_dictionary(data):
    temp_dict = {}
    for key in data.iteritems():
        if key[1] not in temp_dict.keys():
            temp_dict[key[1]] = len(temp_dict)
    return temp_dict

In [9]:
# use clean df with no duplicates
df = df.copy()

# create dictionaries for string variables
city_dict = create_dictionary(df['city'])
state_dict = create_dictionary(df['state'])
country_dict = create_dictionary(df['country'])
cat_dict = create_dictionary(df['categories'])

# map dictionaries to dataframe
df['categories'] = df['categories'].map(cat_dict)
df['country'] = df['country'].map(country_dict)
df['city'] = df['city'].map(city_dict)
df['state'] = df['state'].map(state_dict)
df['target'] = df['target'].map({'failed':0, 'successful':1})

## Get a Subset of Dataframe (500,000 rows)

In [ ]:
df.shape

In [ ]:
df_small = df.sample(n = 500000, random_state=22)
print(df_small.shape)
df_small.head()

## Train/Test Split

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
# create df
traintest = df.copy()

names = traintest.name.unique()
train_names, test_names = train_test_split(names, test_size=0.2, random_state=22)
train = traintest[traintest['name'].isin(train_names)]
test = traintest[traintest['name'].isin(test_names)]

# pick features
target = 'target'
features = ['category', 'campaign_duration_days'
            , 'goal_usd', 'country']



X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train.head()

## Model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

In [ ]:
# %%time
# scores = cross_val_score(model, X_test, y_test, cv=10)
# print('scores:', scores, '\n')
# print('mean:',scores.mean())

# # 83% accuracy

In [ ]:
%%time

y_pred = model.predict(X_test)
len(set(y_pred))
score = accuracy_score(y_pred, y_test)
print(f'accuracy score {score}')

In [ ]:
import pickle
# pickle.dump(model, open('model_rf_sat.pkl', 'wb'))

## NLP

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

nlp = spacy.load('en_core_web_md')

In [11]:
df.head()

,id,campaignName,description,categories,duration,monetaryGoal,country,city,state,date_created,deadline_date,launched_date,state_changed_at,backers_count,usd_pledged,url,target
0,1123889576,mars-on-earth-an-art-residency,Help a fine art photographer continue her proj...,0,27,1000,0,0,0,2015-06-24,2015-10-23,2015-09-26,2015-10-23,53,1884,https://www.kickstarter.com/projects/cassandra...,1
1,1724173143,vulcan-i-rocket-powered-by-3d-printed-engine,Team of undergraduates racing to be the first ...,0,30,15000,0,1,1,2014-05-06,2015-05-21,2015-04-21,2015-05-21,465,21882,https://www.kickstarter.com/projects/105499101...,1
2,707260502,starscraper-the-next-generation-of-suborbital-...,What if we built a rocket that is better than ...,0,31,10000,0,0,0,2014-11-29,2015-01-09,2014-12-09,2015-01-09,294,17176,https://www.kickstarter.com/projects/burpg/sta...,1
3,497637964,students-building-a-near-space-balloon-with-li...,A group of high school students are building a...,0,30,150,0,2,1,2014-11-19,2015-11-26,2015-10-27,2015-11-26,45,970,https://www.kickstarter.com/projects/136362214...,1
4,1546008758,earth-360,Re-inventing the way we look at our planet by ...,0,30,7500,0,3,2,2012-04-11,2012-09-21,2012-08-22,2012-09-21,28,7576,https://www.kickstarter.com/projects/211370922...,1


In [12]:
df.description[0]

'Help a fine art photographer continue her project about space exploration, Mars, and the scientists who are going to make it possible!'

In [ ]:
# create df
traintest = df.copy()

# train/test split
train, test = train_test_split(traintest, test_size=0.2, random_state=22)

# pick features
target = 'target'
features = ['campaignName','description','categories', 'duration'
            , 'monetaryGoal', 'country']


# X_train = train[features]
# y_train = train[target]
# X_test = test[features]
# y_test = test[target]

X_train = train[features][:100000]
y_train = train[target][:100000]
X_test = test[features][:1000]
y_test = test[target][:1000]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [17]:
# functions for tokenizing

punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations]
    return tokens

def clean_text(text):
    return text.strip().lower()

def spacy_vectorizer(sentence):
    
    # spacy preprocessing
    doc = nlp(sentence)
    
    return doc.vector

In [1]:
def nlp_test(train,test):
    
    X_train = train.copy()
    X_test = test.copy()
    
    # instantiate tokenization
#     cleaner = predictors()
#     bow_vector = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))
    tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer)
    
    # tokenize name
    train_name = X_train['campaignName']
#     train_name = cleaner.fit_transform(train_name)
#     train_name = bow_vector.fit_transform(train_name)
    train_name = tfidf_vector.fit_transform(train_name)
    test_name = X_test['campaignName']
#     test_name = cleaner.transform(test_name)
#     test_name = bow_vector.transform(test_name) 
    test_name = tfidf_vector.transform(test_name) 
    
    # create name matrix
    train_name_matrix = train_name.todense()
    test_name_matrix = test_name.todense()
    
    # create df from name matrix
    train_name_matrix_df = pd.DataFrame(train_name_matrix, columns=tfidf_vector.get_feature_names())
    test_name_matrix_df = pd.DataFrame(test_name_matrix, columns=tfidf_vector.get_feature_names())
    
#     # tokenize blurb
#     train_blurb = X_train['blurb']
#     train_blurb = cleaner.fit_transform(train_blurb)
#     train_blurb = bow_vector.fit_transform(train_blurb)
# #     train_blurb = tfidf_vector.fit_transform(train_blurb)
#     test_blurb = X_test['blurb']
#     test_blurb = cleaner.transform(test_blurb)
#     test_blurb = bow_vector.transform(test_blurb) 
# #     test_blurb = tfidf_vector.transform(test_blurb) 
    
#     # create blurb matrix
#     train_blurb_matrix = train_blurb.todense()
#     test_blurb_matrix = test_blurb.todense()
    
#     # create df from blurb matrix
#     train_blurb_matrix_df = pd.DataFrame(train_blurb_matrix, columns=bow_vector.get_feature_names())
#     test_blurb_matrix_df = pd.DataFrame(test_blurb_matrix, columns=bow_vector.get_feature_names())
    
    #get lengths of train and test
    train_len = len(X_train)
    test_len = len(X_test)

    # add id for merge
    X_train['id'] = list(range(train_len))
    train_name_matrix_df['id'] = list(range(train_len))
#     train_blurb_matrix_df['id'] = list(range(train_len))
    X_test['id'] = list(range(test_len))
    test_name_matrix_df['id'] = list(range(test_len))
#     test_blurb_matrix_df['id'] = list(range(test_len))

    # merge train with name matrix
    X_train = X_train.merge(train_name_matrix_df, on='id', how='inner')
    X_test = X_test.merge(test_name_matrix_df, on='id', how='inner')
    
#     # merge train with blurb matrix
#     X_train = X_train.merge(train_blurb_matrix_df, on='id', how='inner')
#     X_test = X_test.merge(test_blurb_matrix_df, on='id', how='inner')
    
#     # drop unnecessary columns
#     X_train = X_train.drop(columns=['name', 'blurb_x', 'id'])
#     X_test = X_test.drop(columns=['name', 'blurb_x', 'id'])
    
    # drop unnecessary columns
    X_train = X_train.drop(columns=['campaignName', 'id'])
    X_test = X_test.drop(columns=['campaignName', 'id'])
    
    assert len(X_train) == len(train)
    assert len(X_test) == len(test)
    
    return X_train, X_test


In [ ]:
# X_train = train.copy()
# X_test = test.copy()
    
# # instantiate tokenization
# cleaner = predictors()
# bow_vector = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))
# tfidf_vector = CountVectorizer(tokenizer=spacy_tokenizer)
    
# # tokenize name
# train_name = X_train['name']
# train_name = cleaner.fit_transform(train_name)
# train_name = bow_vector.fit_transform(train_name)
# # train_name = tfidf_vector.fit_transform(train_name)
# test_name = X_test['name']
# test_name = cleaner.transform(test_name)
# test_name = bow_vector.transform(test_name) 
# # test_name = tfidf_vector.transform(test_name) 
    
# # create name matrix
# train_name_matrix = train_name.todense()
# test_name_matrix = test_name.todense()
    
# # create df from name matrix
# train_name_matrix_df = pd.DataFrame(train_name_matrix, columns=bow_vector.get_feature_names())
# test_name_matrix_df = pd.DataFrame(test_name_matrix, columns=bow_vector.get_feature_names())
    
# # # tokenize blurb
# # train_blurb = X_train['blurb']
# # train_blurb = cleaner.fit_transform(train_blurb)
# # train_blurb = bow_vector.fit_transform(train_blurb)
# # # train_blurb = tfidf_vector.fit_transform(train_blurb)
# # test_blurb = X_test['blurb']
# # test_blurb = cleaner.transform(test_blurb)
# # test_blurb = bow_vector.transform(test_blurb) 
# # # test_blurb = tfidf_vector.transform(test_blurb) 
    
# # # create blurb matrix
# # train_blurb_matrix = train_blurb.todense()
# # test_blurb_matrix = test_blurb.todense()
    
# # # create df from blurb matrix
# # train_blurb_matrix_df = pd.DataFrame(train_blurb_matrix, columns=bow_vector.get_feature_names())
# # test_blurb_matrix_df = pd.DataFrame(test_blurb_matrix, columns=bow_vector.get_feature_names())
    
# #get lengths of train and test
# train_len = len(X_train)
# test_len = len(X_test)

# # add id for merge
# X_train['id'] = list(range(train_len))
# train_name_matrix_df['id'] = list(range(train_len))
# # train_blurb_matrix_df['id'] = list(range(train_len))
# X_test['id'] = list(range(test_len))
# test_name_matrix_df['id'] = list(range(test_len))
# # test_blurb_matrix_df['id'] = list(range(test_len))

# # # merge train with name matrix
# # X_train = X_train.merge(train_name_matrix_df, on='id', how='inner')
# # X_test = X_test.merge(test_name_matrix_df, on='id', how='inner')
    
# #     # merge train with blurb matrix
# #     X_train = X_train.merge(train_blurb_matrix_df, on='id', how='inner')
# #     X_test = X_test.merge(test_blurb_matrix_df, on='id', how='inner')
    
# #     # drop unnecessary columns
# #     X_train = X_train.drop(columns=['name', 'blurb_x', 'id'])
# #     X_test = X_test.drop(columns=['name', 'blurb_x', 'id'])
    
# assert len(X_train) == len(train)
# assert len(X_test) == len(test)
    




In [ ]:
X_train_new, X_test_new = nlp_test(X_train, X_test)

# nlp_stuff(X_train, X_test)
X_train_new.shape, y_train.shape, X_test_new.shape, y_test.shape

In [ ]:
X_train_new.head()

In [ ]:
%%time

model = RandomForestClassifier(1000)
model.fit(X_train_new,y_train)

In [ ]:
%%time

y_pred = model.predict(X_test_new)
len(set(y_pred))
score = accuracy_score(y_pred, y_test)
print(f'accuracy score {score}')

In [ ]:
pip install xgboost

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
%%time

# model
model = XGBClassifier()

# fit model
model.fit(X_train_new,y_train)

# predictions
y_pred = model.predict(X_test_new)

# accuracy
score = accuracy_score(y_pred, y_test)
print(f'accuracy score {score}')

## Heroku Testing

In [4]:
import json
import requests

In [55]:
url = 'http://kickstarter-success.herokuapp.com/'


data = {
  "name": 1,
  "category": 5,
  "blurb": 1,
  "goal_usd": 100000,
  "campaign_duration_days": 30,
  "country": 3
}

data = json.dumps(data)

In [56]:
send_request = requests.post(url, data)
print(send_request)

<Response [200]>


In [57]:
print(send_request.json())

{'results': {'results': 1}}
